# Исследование надежности заемщиков.

### Описание проекта
**Заказчик** — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

**Входные данные от банка** — статистика о платёжеспособности клиентов.

**Результаты исследования** будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** — проверить 4 гипотезы:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

Данные о платёжеспособности клиентов представлены в файле /datasets/data.csv 

Исследование пройдет в **4 этапа:**
1. Изучение общей информации о данных.
2. Предобработка данных.
3. Ответы на поставленные вопросы.
4. Общие выводы. 


### Шаг 1. Обзор данных

In [1]:
#проигнорируем предупреждения -_-
import warnings
warnings.filterwarnings('ignore')

In [2]:
# импорт библиотеки pandas
import pandas as pd

In [3]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('/datasets/data.csv')

In [4]:
# получение первых 10 строк таблицы df
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [5]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице 12 столбцов. Типы данных в столбцах — float, int и object.

* children  — кол-во детей;
* days_employed — трудовой стаж;
* dob_years — возраст;
* education — образование;
* education_id — идентификатор образования;
* family_status — семейное положение;
* family_status_id — идентификатор семейного положения;
* gender — пол;
* income_type — тип дохода;
* debt — долг;
* total_income — суммарный доход;
* purpose цель — (на что кредит).

**Выводы:**
1. В каждой строке таблицы представлены данные о клиенте. Колонки рассказывают о семейном положении клиента, о его доходах, образовании, а также часть колонк описывают суммарный доход клиента, оставшийся долг и цель кредита. 
2. В названиях колонок нарушения стиля нет. 
3. В двух столбцах (days_employed и total_income) есть пропущенные значения.
4. Пристутсвуют аномальные значения. Например, столбце days_employed (трудовой стаж) встречаются отрицательные значения.
5. Не все типы данных подходят для анализа.  

Чтобы двигаться дальше, нужно устранить проблемы в данных.

### Шаг 2.1 Заполнение пропусков

In [6]:
# подсчёт пропусков
display(df.isna().sum()) 


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Таким образом, мы видим, что в 2-х столбцах есть 2174 пропущенных значений. Колонки days_employed и total_income хранят в себе количественные значения. Проверим, какую долю составляют пропущенные значения в каждом из столбцов с пропусками:

In [7]:
#доля пропущенных значений 
total_df = len(df) #всего значений в таблице
missing_values_df = df['days_employed'].isna().sum() #всего пропущенных значений, в обоих столбцах кол-во пропусков одинаковое 
print(f'Доля пропущенных значений в каждом из столбцов - {missing_values_df / total_df:.0%}') 

Доля пропущенных значений в каждом из столбцов - 10%


Пропуски в количественных данных лучше заполнить характерными значениями. Это значения, характеризующие состояние выборки, — набора данных, выбранных для проведения исследования. Чтобы примерно оценить типичные значения выборки подходит медиана.

In [8]:
#заполним пропуски медианой с группировкой по типу дохода (пенсионер, сотрудник и тд)

#заполнение столбца с трудовым стажем
print('Пропуски до:', df['days_employed'].isna().sum()) #кол-во пропусков до заполнения

for income_type in df['income_type'].unique():
    median = df.loc[df['income_type'] == income_type, 'days_employed'].median()
    print(income_type, median) #вывод значения медианы для каждого типа
    df.loc[(df['days_employed'].isna()) & (df['income_type'] == income_type), 'days_employed'] = median

print('Пропуски после:', df['days_employed'].isna().sum()) # проверка кол-ва пропусков после заполнения

Пропуски до: 2174
сотрудник -1574.2028211070854
пенсионер 365213.3062657312
компаньон -1547.3822226779334
госслужащий -2689.3683533043886
безработный 366413.65274420456
предприниматель -520.8480834953765
студент -578.7515535382181
в декрете -3296.7599620220594
Пропуски после: 0


In [9]:
#заполнение столбца с суммарным доходом
print('Пропуски до:', df['total_income'].isna().sum()) #кол-во пропусков до заполнения

for income_type in df['income_type'].unique():
    median = df.loc[df['income_type'] == income_type, 'total_income'].median()
    print(income_type, median) #вывод значения медианы для каждого типа
    df.loc[(df['total_income'].isna()) & (df['income_type'] == income_type), 'total_income'] = median

print('Пропуски после:', df['total_income'].isna().sum()) # проверка кол-ва пропусков после заполнения

Пропуски до: 2174
сотрудник 142594.39684740017
пенсионер 118514.48641164352
компаньон 172357.95096577113
госслужащий 150447.9352830068
безработный 131339.7516762103
предприниматель 499163.1449470857
студент 98201.62531401133
в декрете 53829.13072905995
Пропуски после: 0


In [10]:
# проверочный подсчёт пропусков в таблице после преобразований
display(df.isna().sum()) 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [11]:
#на предыдущих шагах можно было заметить, что в столбце с трудовым штатом присутствуют аномально большие значения и отрицательные
#отрицательного стажа быть не может, поэтому необходимо все привести к положительным значениям
#посомтрим первые 50 строчек столбца
df['days_employed'].head(50)

0      -8437.673028
1      -4024.803754
2      -5623.422610
3      -4124.747207
4     340266.072047
5       -926.185831
6      -2879.202052
7       -152.779569
8      -6929.865299
9      -2188.756445
10     -4171.483647
11      -792.701887
12    365213.306266
13     -1846.641941
14     -1844.956182
15      -972.364419
16     -1719.934226
17     -2369.999720
18    400281.136913
19    -10038.818549
20     -1311.604166
21      -253.685166
22     -1766.644138
23      -272.981385
24    338551.952911
25    363548.489348
26     -2689.368353
27      -529.191635
28      -717.274324
29    365213.306266
30    335581.668515
31     -1682.083438
32     -4649.910832
33     -1548.637544
34     -4488.067031
35    394021.072184
36      -176.216688
37     -6448.810860
38      -597.881827
39      -650.587796
40     -1030.362606
41     -2689.368353
42     -1257.496190
43     -4375.681384
44     -1362.041728
45     -1039.451741
46     -2262.712304
47     -2689.137274
48     -3341.067886
49     -1181.443228


Можно сделать вывод, что верные значения представлены в днях и находятся в диапозоне от 0 до 15000 дней (примерно от 0 до 41 года трудовой стаж - это норма). Все значения, что больше 15000 скорее всего представлены в часах.  Скорее всего, в данных встречается информация из двух систем. Одна из них считает стаж в часах и "наоборот".
Переведем все к единому измерению в днях. 

In [12]:
df['days_employed'] = df['days_employed'].apply(abs) # берет значения по модулю, избавляемся от отрицательных значений
df.loc[df['days_employed'] > 15000, 'days_employed'] = df.loc[df['days_employed'] > 15000, 'days_employed'] / 24 #переводим часы в дни
df['days_employed'].head(50) #проверяем результат

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4     14177.753002
5       926.185831
6      2879.202052
7       152.779569
8      6929.865299
9      2188.756445
10     4171.483647
11      792.701887
12    15217.221094
13     1846.641941
14     1844.956182
15      972.364419
16     1719.934226
17     2369.999720
18    16678.380705
19    10038.818549
20     1311.604166
21      253.685166
22     1766.644138
23      272.981385
24    14106.331371
25    15147.853723
26     2689.368353
27      529.191635
28      717.274324
29    15217.221094
30    13982.569521
31     1682.083438
32     4649.910832
33     1548.637544
34     4488.067031
35    16417.544674
36      176.216688
37     6448.810860
38      597.881827
39      650.587796
40     1030.362606
41     2689.368353
42     1257.496190
43     4375.681384
44     1362.041728
45     1039.451741
46     2262.712304
47     2689.137274
48     3341.067886
49     1181.443228
Name: days_employed, dtype: float64

В столбце с количеством детей присутствуют знаяения 20 и -1. Доля аномалий мала и ни на что не повлияет, поэтому исключим эти строки.

In [13]:
df = df[df['children'] != 20] #убираем строки со значением 20
df = df[df['children'] != -1] #убираем строки со значением -1
df['children'].unique()
        

array([1, 0, 3, 2, 4, 5])

### Шаг 2.3. Изменение типов данных.

In [14]:
#заменим вещественный тип данных в столбце total_income на целочисленный
df['total_income'] = df['total_income'].astype('int')
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.4. Удаление дубликатов.

In [15]:
#посомтрим неявные дубликаты в столбце с образованием education 
df['education'].value_counts()

среднее                13667
высшее                  4698
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      665
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [16]:
df['education'] = df['education'].str.lower() #переведем все значения в столбце к нижнему регистру
df['education'].value_counts() 

среднее                15136
высшее                  5237
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

In [17]:
#посомтрим неявные дубликаты в столбце с образованием education 
df['education'].value_counts()

среднее                15136
высшее                  5237
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

Итак. 
Все пропуски заполнены. 
Аномальные значения обработаны. 
Неявных дублей больше нет (привели данные к единому регистру)
Тепрь посмотрим явные дубли и удалим их.

In [18]:
df.duplicated().sum() #посчитаем явные дубликаты 

71

In [19]:
df = df.drop_duplicates() #удалим явные дубликаты 
df.duplicated().sum() #посчитаем явные дубликаты после удаления 

0

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [20]:
#создадим дополнительные словари для столбцов с образованием и семейным положением
education_dict = df[['education','education_id']]
education_dict['education'] = education_dict['education'].str.lower()
education_dict = education_dict.drop_duplicates()

family_status_dict = df[['family_status','family_status_id']]
family_status_dict['family_status'] = family_status_dict['family_status'].str.lower()
family_status_dict = family_status_dict.drop_duplicates()

#удалим из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id.
df.drop(['education'], axis=1, inplace=True) 
df.drop(['family_status'], axis=1, inplace=True) 



In [21]:
df.head() #проверка удаления столбцов

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

In [22]:
#напишем функцию, которая присвоит клиентам категорию от А до Е в зависимости от их дохода
# на основании диапазонов, указанных ниже, создайте столбец total_income_category с категориями:
#0–30000 — 'E';
#30001–50000 — 'D';
#50001–200000 — 'C';
#200001–1000000 — 'B';
#1000001 и выше — 'A'.
def total_income_group(total_income):
    if total_income <= 30000:
        return 'E'
    if 30001 <= total_income <= 50000:
        return 'D'
    if 50001 <= total_income <= 200000:
        return 'C'
    if 200001 <= total_income <= 1000000:
        return 'B'
    return 'A'
df['total_income_category'] = df['total_income'].apply(total_income_group) #вызываем функцию для каждого значения из столбца с доходами
df.head() #смотрим результат

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


In [23]:
df['purpose'].value_counts()

свадьба                                   790
на проведение свадьбы                     763
сыграть свадьбу                           760
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с жильем                         647
операции с коммерческой недвижимостью     645
жилье                                     641
покупка жилья                             640
покупка жилья для семьи                   637
недвижимость                              631
строительство собственной недвижимости    628
операции со своей недвижимостью           623
строительство жилой недвижимости          620
покупка своего жилья                      619
строительство недвижимости                619
покупка недвижимости                      616
ремонт жилью                              604
покупка жилой недвижимости                602
на покупку своего автомобиля              504
заняться высшим образованием      

### Шаг 2.7. Категоризация целей кредита.

In [24]:
#напишем функцию, которая считает значение в каждой строке столбца purpose и в зависимости от его значения присвоит тип операции 
def purpose_category(row):
    purpose_category = row['purpose']
    car = 'автомобил' #ключевое слово, по которому можно определить тип. Ниже по аналогии 
    wedding = 'свадьб'
    education = 'образован'
    
    if car in purpose_category:
        return 'операции с автомобилем'
    if education in purpose_category:
        return 'получение образования'
    if wedding in purpose_category:
        return 'проведение свадьбы'
    return 'операции с недвижимостью'

df['purpose_category'] = df.apply(purpose_category, axis=1)
df.head(10) #смотрим результат

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [25]:
debt_by_children = df.pivot_table(index = 'children', columns = 'debt', values = 'total_income', aggfunc = 'count').fillna(0)
debt_by_children.columns = ['Нет задолженности', 'Задолженность']
debt_by_children['% задолженности'] = debt_by_children['Задолженность'] / (debt_by_children['Задолженность'] + debt_by_children['Нет задолженности'])
display(debt_by_children.style.format({'% задолженности': '{:.2%}', 'Задолженность': '{:.0f}'}))

,Нет задолженности,Задолженность,% задолженности
children,,,
0,13028.000000,1063,7.54%
1,4364.000000,444,9.23%
2,1858.000000,194,9.45%
3,303.000000,27,8.18%
4,37.000000,4,9.76%
5,9.000000,0,0.00%


##### Вывод 1:

По итоговой таблице можно сделать следующие выводы:
1. У клиентов, у которых 5 детей нет задолжностей вообще. 
2. От 1 до 4 детей процент задолжности примерно одинаковый 8-9%.
3. У клиентов без детей этот процент чуть меньше - 7,5%. 

То есть кредиты возвращают лучше всего многодетные семьи с 5 детьми. На втором месте люди без детей. 
Количество детей в промежутке от 1 до 4 никак ни влияет на возврат долгов. 


##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
debt_by_family = df.pivot_table(index = 'family_status_id', columns = 'debt', values = 'total_income', aggfunc = 'count').fillna(0)
debt_by_family.columns = ['Нет задолженности', 'Задолженность']
debt_by_family['% задолженности'] = debt_by_family['Задолженность'] / (debt_by_family['Задолженность'] + debt_by_family['Нет задолженности'])
debt_by_family_report = family_status_dict.merge(debt_by_family, on = 'family_status_id')
display(debt_by_family_report.style.format({'% задолженности': '{:.2%}', 'Задолженность': '{:.0f}'}))

,family_status,family_status_id,Нет задолженности,Задолженность,% задолженности
0,женат / замужем,0,11334,927,7.56%
1,гражданский брак,1,3749,385,9.31%
2,вдовец / вдова,2,888,63,6.62%
3,в разводе,3,1105,84,7.06%
4,не женат / не замужем,4,2523,273,9.76%


##### Вывод 2:

По сводной таблице можно сделать следующие выводы:
1. Больше всего долгов в процентном соотношении у людей, которые не состоят в браке вообще или находятся в гражданском браке - в таких категориях примерно 9,5% долгов. 
2. Лучше всего платят по кредитам вдовцы. У них меньше всего процент задолжностей - 6,6%. 
3. У клиентов в браке или разводе процент задолжности средний - примерно 7%.

Таким образом, семейное положение незначительно, но влияет на платежеспособность клиентов. 

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
debt_by_total_income = df.pivot_table(index = 'total_income_category', columns = 'debt', values = 'total_income', aggfunc = 'count').fillna(0)
debt_by_total_income.columns = ['Нет задолженности', 'Задолженность']
debt_by_total_income['% задолженности'] = debt_by_total_income['Задолженность'] / (debt_by_total_income['Задолженность'] + debt_by_total_income['Нет задолженности'])
display(debt_by_total_income.style.format({'% задолженности': '{:.2%}', 'Задолженность': '{:.0f}'}))

,Нет задолженности,Задолженность,% задолженности
total_income_category,,,
A,23,2,8.00%
B,4660,354,7.06%
C,14568,1353,8.50%
D,328,21,6.02%
E,20,2,9.09%


##### Вывод 3:

По итоговой таблице можно сделать следующие вывод, что суммарный доход не влияет на возврат долгов по кредиту. 
Например, клиенты с самым высоким доходом "А" имеют такой же процент задолженности, как и клиенты со средним достатком категории "С". 
Явной закономерности не наблюдается. 

Однако, можно заметить, что сильно различается количество выданных займов. 
Больше всего кредитов берут клиенты категории "С" - среднего дохода. 
А к категориям "А" - высокий доход и "Е" - низкий доход, количество выданных кредитов значительно уменьшается. 

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [28]:
debt_by_purpose = df.pivot_table(index = 'purpose_category', columns = 'debt', values = 'purpose', aggfunc = 'count').fillna(0)
debt_by_purpose.columns = ['Нет задолженности', 'Задолженность']
debt_by_purpose['% задолженности'] = debt_by_purpose['Задолженность'] / (debt_by_purpose['Задолженность'] + debt_by_purpose['Нет задолженности'])
display(debt_by_purpose.style.format({'% задолженности': '{:.2%}', 'Задолженность': '{:.0f}'}))

,Нет задолженности,Задолженность,% задолженности
purpose_category,,,
операции с автомобилем,3879,400,9.35%
операции с недвижимостью,9971,780,7.26%
получение образования,3619,369,9.25%
проведение свадьбы,2130,183,7.91%


##### Вывод 4:

По сводной таблицу можно сделать следующие выводы:
1. Клиенты лучше платят по кредитам, взятым на операции с недвижимостью или проведение свадьбы. Процент задолженности у таких составляет около 7,5%.
2. Хуже возвращают долги по операциям с автомобилем или образованием. Здесть процент задолженности составляет чуть больше 9%. 



## Общий вывод:

Кредитным отьделом банка кредитный отдел банка нам была поставлена главная цель - выяснить, какие факторы влияют на факт погашения кредита в срок.

Заказчиком были предоставлены входные данные — статистика о платёжеспособности клиентов.
Целью было проверить 4 гипотезы:

- Есть ли зависимость между количеством детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

Основное исследование прошло в 4 этапа:

#### Изучение общей информации о данных.
На данном этапе мы олучили общую информации о данных в таблице и увидели, с чем предстоит иметь дело.

Основные проблемы, которые были выявлены: 
1. В двух столбцах (days_employed и total_income) были пропущенны значения.
2. Пристутсвовали аномальные значения. Например, столбце days_employed (трудовой стаж) - отрицательные значения.
3. Не все типы данных подходили для анализа.

#### Предобработка данных.
На втором этапе мы подготовили решили проблемы, выявленные на первом этапе и подготовили данные ддля дальнейшего анализа.
А именно:
1. Заполненили пропуски.
2. Проверили данные на аномалии и исправили их.
3. Избавились от некорректных данных, которые не повлияют на дальнейший анализ.
4. Измененили типы данных на те, которые подойдут для дальнейшего анализа. 
5. Нашли и обработали неявные дубликаты. (привели данные к единому регистру)
6. Удалили явные дубликаты. 
7. Сформировали дополнительные датафреймы словарей и декомпозировали исходный датафрейм для удобства анализа. 
8. Для удобства анализа категоризировали клиентов:
- в зависимости от их дохода (от А до Е);
- в зависимости от целей кредита (свадьба, недвижимость, образование, авто).

#### Ответы на поставленные вопросы.
На данном третьем этапе мы посторили сводные таблицы и проверили четыре гипотезы. Краткие выводы по каждому вопросу представлены ниже.

1. Наличие и количество детей по-разному влияют на платежеспособность клиентов. Первая гипотеза подтвердилась.

2. Семейное положение также влияет на наличие задолженностей по кредиту, вторая гипотеза подтвердилась.

3. Зависимости между уровнем дохода и возвратом кредита в срок установлено не было. Вопреки ожиданиям люди с большим доходом платят кредит также, как и со средним. Однако берут их значительно реже как клиенты категории "А", так и категории "Е". Третья гипотеза не подтвердилась. 

4. Цель кредита относительно влияют на погашение кредита. Из 4 целей были выделены 2 группы, в которых отличается процент задолженностей. 


#### Общий вывод
Последний, четвертый этап посвящен обобщению результатов исследования и рекомендаций заказчику на основе проведенного анализа. 

Итак, можно сделать вывод, что при построении модели кредитного скоринга банку следует учитывать наличие и количество детей (меньше долгов у многодетных семей (5 детей) и у кредиторов без детей), а также частично на семейное положение кредитора.
Также стоит брать во внимание цели кредита (клиенты лучше платят по кредитам, взятым на операции с недвижимостью или проведение свадьбы).

Зависимости между уровнем дохода и возвратом кредита в срок установлено не было.

